In [1]:
%load_ext autoreload
%autoreload 2
import tichu_rustipy as tr
import numpy as np
from IPython.display import display, HTML
import pickle

def display_colored_hand(hand):
    hand_str = tr.print_hand(hand)
    # Convert ANSI escape codes to HTML
    hand_str = (hand_str
        .replace('\x1b[31m', '<span style="color: red">')
        .replace('\x1b[32m', '<span style="color: green">')
        .replace('\x1b[33m', '<span style="color: yellow">')
        .replace('\x1b[34m', '<span style="color: dodgerblue ">')
        .replace('\x1b[0m', '</span>')
    )
    display(HTML(hand_str))

In [2]:
db = tr.BSWSimple("../tichu_rust/bsw_filtered.db")
display_colored_hand(db.get_round(0)[0].first_14)
db.len()

400881

In [ ]:
for x in tr.get_legal_incoming_card_combinations(db.get_round(167)[1].first_14, label_num_to_incoming_card_combination[7]):
    display_colored_hand(tr.print_hand(x))

In [ ]:
display_colored_hand(tr.print_hand(db.get_round(167)[1].first_14))
print(label_num_to_incoming_card_combination[64])
print(label_num_to_incoming_card_combination[100])
print(label_num_to_incoming_card_combination[246])
print(label_num_to_incoming_card_combination[7])